In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import beta, binom
import itertools
from ipywidgets import interact, interactive


import hashlib
%matplotlib inline

sns.set(style="dark")
plt.style.use("ggplot")

import pymc3 as pm

In [2]:
happy = pd.read_csv('World Happiness Report.csv')
life = pd.read_csv('Life Expectancy Data.csv')

In [3]:
happy = happy.rename(columns={"Country Name": "Country"})

In [4]:
merged = life.merge(happy, how = 'inner', on = ['Country', 'Year'])

In [5]:
columns = ['Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling',
       'Life Ladder', 'Log GDP Per Capita', 'Social Support',
       'Healthy Life Expectancy At Birth', 'Freedom To Make Life Choices',
       'Generosity', 'Perceptions Of Corruption', 'Positive Affect',
       'Negative Affect', 'Confidence In National Government']

In [6]:
merged = merged.dropna()
merged = merged.drop('GDP', axis=1)

In [7]:
Status_Indicator = merged.Status.map(dict(Developing=0, Developed=1))

In [8]:
merged['Status Indicator'] = Status_Indicator

In [9]:
grouped = merged.groupby('Country').mean()
grouped.head()

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,...,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,Status Indicator
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,2011.500000,59.875000,274.500000,70.375000,0.012500,58.202410,64.875000,1790.625000,17.400000,95.750000,...,7.589941,0.515188,51.550000,0.562421,0.141376,0.815038,0.504555,0.286646,0.427282,0.0
Albania,2012.200000,77.060000,52.200000,0.200000,4.918000,333.858237,98.600000,6.200000,55.840000,0.800000,...,9.305332,0.726059,68.216002,0.640089,-0.083702,0.870555,0.547012,0.308215,0.401887,0.0
Argentina,2010.000000,75.588889,98.888889,9.444444,8.163333,1061.475672,81.777778,2.888889,53.811111,10.666667,...,10.044856,0.906328,66.300001,0.719619,-0.147993,0.842842,0.754868,0.263281,0.411542,0.0
Armenia,2010.000000,73.744444,104.444444,1.000000,3.981111,203.076666,90.777778,18.111111,41.422222,1.000000,...,9.177718,0.703882,64.800000,0.496155,-0.217615,0.878101,0.466998,0.431185,0.335829,0.0
Australia,2010.714286,82.000000,55.285714,1.000000,10.250000,9071.594024,92.142857,147.571429,64.328571,1.142857,...,10.728655,0.947152,70.257142,0.924931,0.310600,0.419014,0.745028,0.212288,0.521874,1.0


In [10]:
grouped.mean()

Year                                 2.010646e+03
Life expectancy                      7.009000e+01
Adult Mortality                      1.583724e+02
infant deaths                        3.773170e+01
Alcohol                              4.726366e+00
percentage expenditure               9.270263e+02
Hepatitis B                          8.020482e+01
Measles                              1.495915e+03
 BMI                                 3.818169e+01
under-five deaths                    5.226046e+01
Polio                                8.378040e+01
Total expenditure                    6.318354e+00
Diphtheria                           8.534371e+01
 HIV/AIDS                            1.446300e+00
Population                           1.726651e+07
 thinness  1-19 years                4.731854e+00
 thinness 5-9 years                  4.779330e+00
Income composition of resources      6.605828e-01
Schooling                            1.250156e+01
Life Ladder                          5.233142e+00


OH NOTES
- sounds more like Bayesian GLM rather than Hierarchical
- could use GLMs as a tool
- Bayesian GLM Non-Parametric and Some Parametric Methods
- Predicting Happiness?
- Doesn't make sense to use Bern bc theta is a random variable between 0 and 1
- you're predicting based on the parameters, predicting the probability that a country is developed, but if you make it bern
- Stop here and write up the report
- Not sure if we can do multipple observed

In [11]:
def betaParams(mean, std):
    var = std ** 2
    
    alpha = ((1 - mu) / var - (1 / mu)) * (mu ** 2)
    beta = alpha * (1 / mu - 1)
    
    return list(alpha, beta)

In [12]:
pi = sum(grouped['Status Indicator'])/len(grouped)      # this is a placeholder, proportion of developed
sigma = 0.4    # this is a placeholder, use domain knowledge or find a way to justify
mu_gdp = 9.075154 #explain how we used domain knowledge
sigma_gdp = 1.165183 

In [48]:
with pm.Model() as develop_prediction:
    
    # is country i developed or not?
    y = pm.Bernoulli('y', p=pi)
    
    # mean gdp per group
    lamda_2 = pm.Poisson('lamda_2', mu_expectancy, shape=2) # life expectancy, change this
    lamda = pm.Poisson('lamda', mu=mu_a_mort, shape=2) # adult mortality
    pm.Poisson() #infant deaths
    pm.exponential # percentage expenditure
    BMI_mu = pm.Uniform('bmi') # BMI 
    exponential #under-five deaths
    mu = pm.Normal('mu', mu=mu_gdp, sigma=sigma_gdp, shape=2) # gdp log
    pm.Poisson # population 
    pm.Beta #Income composition of resources
    pm.Gamma # Schooling
    ladder_mu = pm.Uniform('ladder', lower=0, upper=10, shape=2) # Life Ladder
    pm.Beta # Social Support
    pm.Gamma # healthy life 
    pm.Beta #Freedom To Make Life Choices
    pm.Normal #Generosity 
    pm.Beta #Perceptions Of Corruption
    pm.Beta #Confidence In National Government
    
    
    
    gdp_mean = mu[y]
    other = mu_1[y]
    
    # putting the model together
    x = pm.Normal('x', mu=(gdp_mean*0.5 + other*0.5), sigma=sigma, observed=merged['Log GDP Per Capita'])
    
    trace = pm.sample(2000, chains=2, tune=1000, return_inferencedata=True)

Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>BinaryGibbsMetropolis: [y]
>NUTS: [mu_1, mu]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 20 seconds.
/Users/cassandracalciano/opt/anaconda3/lib/python3.9/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


In [47]:
trace.posterior

<xarray.Dataset>
Dimensions:     (chain: 1, draw: 2000, mu_dim_0: 2, mu_1_dim_0: 2)
Coordinates:
  * chain       (chain) int64 0
  * draw        (draw) int64 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999
  * mu_dim_0    (mu_dim_0) int64 0 1
  * mu_1_dim_0  (mu_1_dim_0) int64 0 1
Data variables:
    y           (chain, draw) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    mu          (chain, draw, mu_dim_0) float64 9.563 17.58 ... 7.946 17.44
    mu_1        (chain, draw, mu_1_dim_0) float64 0.179 0.7051 ... 0.1338 0.8904
Attributes:
    created_at:                 2023-05-05T19:28:04.376810
    arviz_version:              0.12.1
    inference_library:          pymc3
    inference_library_version:  3.11.5
    sampling_time:              4.98589015007019
    tuning_steps:               1000

In [ ]:
# use posterior predictive to see if this is a reasonable model